In [1]:
import numpy as np

In [2]:
identifier = 'Sepsis_WGAN_clip'

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [4]:
real_data_Sepsis = np.load('./data/'+identifier+'real_data.npy')

In [5]:
fake_data_Sepsis = np.load('./data/'+identifier+'fake_data.npy')

In [6]:
real_data_Sepsis.shape

(2627, 30, 5)

In [7]:
fake_data_Sepsis.shape

(2184, 30, 5)

In [8]:
identifier = 'NonSepsis_WGAN_clip'

In [9]:
real_data_NonSepsis = np.load('./data/'+identifier+'real_data.npy')

In [10]:
fake_data_NonSepsis = np.load('./data/'+identifier+'fake_data.npy')

In [11]:
real_data_NonSepsis.shape

(1314, 30, 5)

In [12]:
fake_data_NonSepsis.shape

(1092, 30, 5)

## Train on Real

In [13]:
## 0 - Non-Sepsis
## 1 - Sepsis

In [14]:
real_data_X = np.vstack((real_data_NonSepsis,real_data_Sepsis))
real_data_X.shape

(3941, 30, 5)

In [15]:
real_data_Y = np.concatenate([np.zeros(real_data_NonSepsis.shape[0]),np.ones(real_data_Sepsis.shape[0])])
real_data_Y.shape

(3941,)

In [16]:
seq_len = real_data_Sepsis.shape[1]
feature_len = real_data_Sepsis.shape[2]

In [17]:
print(seq_len)
print(feature_len)

30
5


## Model

In [18]:
import tensorflow as tf

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [20]:
model = Sequential()
model.add(LSTM(100, input_shape=(seq_len, feature_len)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               42400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 42,501
Trainable params: 42,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_val, y_train, y_val = train_test_split(real_data_X, real_data_Y, test_size=0.2, random_state=42)

In [24]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(3152, 30, 5)
(789, 30, 5)
(3152,)
(789,)


In [25]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Train on 3152 samples, validate on 789 samples
Epoch 1/10
3152/3152 [==============================] - 26s 8ms/step - loss: 0.6092 - acc: 0.6640 - val_loss: 0.5850 - val_acc: 0.6616
Epoch 2/10
3152/3152 [==============================] - 16s 5ms/step - loss: 0.5716 - acc: 0.6989 - val_loss: 0.5636 - val_acc: 0.7047
Epoch 3/10
3152/3152 [==============================] - 13s 4ms/step - loss: 0.5516 - acc: 0.7145 - val_loss: 0.5393 - val_acc: 0.7199
Epoch 4/10
3152/3152 [==============================] - 14s 4ms/step - loss: 0.5313 - acc: 0.7338 - val_loss: 0.5193 - val_acc: 0.7452
Epoch 5/10
3152/3152 [==============================] - 10s 3ms/step - loss: 0.5112 - acc: 0.7424 - val_loss: 0.5191 - val_acc: 0.7402
Epoch 6/10
3152/3152 [==============================] - 16s 5ms/step - loss: 0.4931 - acc: 0.7586 - val_loss: 0.4964 - val_acc: 0.7681
Epoch 7/10
3152/3152 [==============================] - 16s 5ms/step - loss: 0.4598 - acc: 0.7798 - val_loss: 0.4809 - val_acc: 0.7655
Epoch 8/

## Test on Synthetic

In [26]:
fake_data_X = np.vstack((fake_data_NonSepsis,fake_data_Sepsis))
fake_data_X.shape

(3276, 30, 5)

In [27]:
fake_data_Y = np.concatenate([np.zeros(fake_data_NonSepsis.shape[0]),np.ones(fake_data_Sepsis.shape[0])])
fake_data_Y.shape

(3276,)

In [28]:
scores = model.evaluate(real_data_X, real_data_Y, verbose=1)

3941/3941 [==============================] - 5s 1ms/step


In [29]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.05%
